In [1]:
import os
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.aio import SearchClient
from azure.search.documents.models import QueryType, Vector
from dotenv import load_dotenv
import json
import re

import sys
sys.path.append('../app/backend')
from core.modelhelper import get_token_limit
from core.messagebuilder import MessageBuilder
from core.language_detector import detect_language
from text import nonewlines

with open('../.azure/config.json', 'r') as config_file:
    config_data = json.load(config_file)
dotenv_path = f'../.azure/{config_data["defaultEnvironment"]}/.env'
load_dotenv(dotenv_path) # Load environment variables from .env file

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT") 
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER") 
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") 
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") 
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL")

KB_FIELDS_CONTENT = "content"
KB_FIELDS_CATEGORY = "category"
KB_FIELDS_SOURCEPAGE = "sourcepage"

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service

# Used by the OpenAI SDK
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"

"# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = \"azure\" instead\n"
#openai.api_type = "azure_ad"
openai.api_key = os.environ.get("OPENAI_API_KEY")#azure_credential.get_token("https://cognitiveservices.azure.com/.default").token

# Set up clients for Cognitive Search and Storage
admin_key = os.environ.get("AZURE_COGNITIVE_SEARCH_KEY")

search_client = SearchClient(
    endpoint= os.environ.get("AZURE_COGNITIVE_SEARCH_ENDPOINT"),
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(admin_key))

In [2]:
import sys 
sys.path.append('../app/backend')
from approaches.chatreadretrieveread import ChatReadRetrieveReadApproach

In [3]:
chat = ChatReadRetrieveReadApproach(search_client, 
                                        AZURE_OPENAI_CHATGPT_DEPLOYMENT,
                                        "gpt-35-turbo", 
                                        AZURE_OPENAI_EMB_DEPLOYMENT,
                                        "sourcepage", 
                                        "content")

In [27]:
request = {'history': [{'user': "How to activate parental control? WUU28TA8"}],
  'approach': 'rrr', 'overrides': 
  {'retrieval_mode': 'hybrid', 'semantic_ranker': True, 'semantic_captions': False, 'top': 5, 'suggest_followup_questions': False}}


In [28]:
overrides = request["overrides"]
history = request["history"]

In [29]:
r = await chat.run(request["history"], request.get("overrides") or {})
r["answer"], r["data_points"]

prompt for query generation: Generate search query for: How to activate parental control? WUU28TA8

Message from chat history for product filter generation: [{'role': 'system', 'content': 'Based on the entire conversation history below:\nYou have two task.\n1: Accurately identify the product id.\nEven if it\'s from previous messages in the conversation. \nIf there are multiple product ids, return the most recent one.\nEnsure that the last question is still referring to the correct product id.\nProduct ids are made of alpha-numeric characters like "SMS6TCI00E", "WUU28TA8". \nIf the product ID isn\'t clear or not mentioned, try to understand if the question is about a dish-washer or a washing machine. \nIf it\'s impossible to determine return "unknown".\n\n2: Understand if the last user question is about a washing-machine or a dish-washer.\nIf it\'s impossible to determine return "unknown". Only possible values are "WASHING MACHINE", "DISH WASHER" and "unknown".\n\nEnsure you return the 

('To activate parental control on the WUU28TA8 washing machine, press and hold the two 8 3 Sek. buttons for approximately 3 seconds [WUU28TA8_en-us_washing-machine_product-manual-22.pdf]. The display will show 8, indicating that the controls are locked. The childproof lock remains active even after the appliance has been switched off and in the event of a power failure [WUU28TA8_en-us_washing-machine_product-manual-31.pdf].',
 ['WUU28TA8_en-us_washing-machine_product-manual-3.pdf:  1.2 Intended use Only use this appliance: ¡ For washing machine-washable fabrics and hand-washable wool in accordance with the care label. ¡ With tap water and commercially available detergents and care products suitable for washing machines. ¡ In private households and in enclosed spaces in a domestic en- vironment. ¡ Up to an altitude of max. 4000 m above sea level. 1.3 Restriction on user group This appliance may be used by children aged 8 or over and by people who have reduced physical, sensory or mental

In [22]:
r["data_points"]

["WUU28TA8_en-us_washing-machine_product-manual-11.pdf: 3 Requirements for the in- stallation location WARNING Risk of electric shock! The appliance contains live parts. Touching live parts is dangerous. ▶ Do not operate the appliance without a cover cap. WARNING Risk of injury! When using the appliance on a base, the appliance may tip over. ▶ Always secure the feet of the appli- ance to a base using the manufac- turer's fixing brackets → Page 28 before using the appliance for the first time. Installation and connection ATTENTION! If residual water in the appliance freezes, this may cause damage to the appliance. ▶ Do not install or operate this appli- ance anywhere that is liable to ex- perience frost or outdoors. If the appliance is tilted at an angle greater than 40°, residual water may leak out of the appliance and cause material damage. ▶ Tilt the appliance carefully. ▶ Transport the appliance in an up- right position. Installation loca- Requirements tion Base Secure the appli- an

In [23]:
print(r["answer"])

There could be several reasons for water inside your WUU28TA8 washing machine [WUU28TA8_en-us_washing-machine_product-manual-37.pdf]:
- The filter in the water inflow is blocked: Clean the water inflow filter.
- The tap is turned off: Turn on the tap.
- The water inlet hose is kinked or jammed: Ensure that the water inlet hose is not kinked or jammed.
- Water level measurement system is defective: Call the after-sales service.
- E:35 / -10: There is water in the base trough: Turn off the water tap and call after-sales service.
